In [1]:
import numpy as np
import pandas as pd
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from string import punctuation
punctuation = list(punctuation)
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_csv('/kaggle/input/cleaned-data-for-the-chatbot-collected-from-movies/dialogs_expanded.csv')
data

,Unnamed: 0,question,answer,question_as_int,answer_as_int,question_len,answer_len
0,1,"Well, I thought we'd start with pronunciation,...",Not the hacking and gagging and spitting part....,"[54, 67, 74, 74, 12, 1, 40, 1, 82, 70, 77, 83,...","[45, 77, 82, 1, 82, 70, 67, 1, 70, 63, 65, 73,...",71,55
1,2,Not the hacking and gagging and spitting part....,Okay... then how 'bout we try out some French ...,"[45, 77, 82, 1, 82, 70, 67, 1, 70, 63, 65, 73,...","[46, 73, 63, 87, 14, 14, 14, 1, 82, 70, 67, 76...",55,73
2,3,You're asking me out. That's so cute. What's ...,Forget it.,"[56, 77, 83, 8, 80, 67, 1, 63, 81, 73, 71, 76,...","[37, 77, 80, 69, 67, 82, 1, 71, 82, 14]",62,10
3,4,"No, no, it's my fault -- we didn't have a prop...",Cameron.,"[45, 77, 12, 1, 76, 77, 12, 1, 71, 82, 8, 81, ...","[34, 63, 75, 67, 80, 77, 76, 14]",65,8
4,9,"Gosh, if only we could find Kat a boyfriend...",Let me see what I can do.,"[38, 77, 81, 70, 12, 1, 71, 68, 1, 77, 76, 74,...","[43, 67, 82, 1, 75, 67, 1, 81, 67, 67, 1, 85, ...",46,25
...,...,...,...,...,...,...,...
139404,221608,Well that one. The one who keeps looking at me.,ft could be you flatter yourself CoghilL It's ...,"[54, 67, 74, 74, 1, 82, 70, 63, 82, 1, 77, 76,...","[68, 82, 1, 65, 77, 83, 74, 66, 1, 64, 67, 1, ...",47,59
139405,221609,Choose your targets men. That's right Watch th...,Keep steady. You're the best shots of the Twen...,"[34, 70, 77, 77, 81, 67, 1, 87, 77, 83, 80, 1,...","[42, 67, 67, 78, 1, 81, 82, 67, 63, 66, 87, 14...",61,85
139406,221610,Colonel Durnford... William Vereker. I hear yo...,"Good ones, yes, Mr Vereker. Gentlemen who can ...","[34, 77, 74, 77, 76, 67, 74, 1, 35, 83, 80, 76...","[38, 77, 77, 66, 1, 77, 76, 67, 81, 12, 1, 87,...",74,60
139407,221611,"Your orders, Mr Vereker?",I'm to take the Sikali with the main column to...,"[56, 77, 83, 80, 1, 77, 80, 66, 67, 80, 81, 12...","[40, 8, 75, 1, 82, 77, 1, 82, 63, 73, 67, 1, 8...",24,56


In [3]:
data.isnull().sum()

Unnamed: 0         0
question           0
answer             0
question_as_int    0
answer_as_int      0
question_len       0
answer_len         0
dtype: int64

In [4]:
# Modify the form of some words
replace_list = {r"'m": ' am',
                r"'re": ' are',
                r"let’s": 'let us',
                r"'s":  ' is',
                r"'ve": ' have',
                r"can't": 'can not',
                r"cannot": 'can not',
                r"shan’t": 'shall not',
                r"n't": ' not',
                r"'d": ' would',
                r"'ll": ' will',
                r"'scuse": 'excuse',
                ',': ' ,',
                '.': ' .',
                '!': ' !',
                '?': ' ?',
                '\s+': ' '}
def clean_text(text):
    text = text.lower()
    for s in replace_list:
        text = text.replace(s, replace_list[s])
    text = ' '.join(text.split())
    return text
# Apply the clean_text function
data['question'] = data['question'].apply(lambda p: clean_text(p))
data['answer'] = data['answer'].apply(lambda p: clean_text(p))

In [5]:
data_ques = data['question']
data_ans = data['answer']

In [6]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [7]:
def preprocessing_data(data):
    #tokenization
    data_tokens = [word_tokenize(data[i]) for i in range(len(data))]
    
    #remove stopwords
    data_cleaned = []
    for i in range(len(data_tokens)):
        data_cleaned.append(' '.join([lemmatizer.lemmatize(word.lower(), pos="v") for word in data_tokens[i] 
                             if not word.lower() in stop_words and word not in punctuation]))  
    return data_cleaned

In [8]:
data_ques = preprocessing_data(data_ques)
data_ans = preprocessing_data(data_ans)

data_ques = np.array(data_ques)
data_ans = np.array(data_ans)

In [9]:
zip_data =  list(zip(data_ques, data_ans))
lines = pd.DataFrame(zip_data, columns = ['questions' , 'answers']) 
lines.head()

,questions,answers
0,well think would start pronunciation okay,hack gag spit part please
1,hack gag spit part please,okay 'bout try french cuisine saturday night
2,ask cute name,forget
3,fault -- proper introduction --,cameron
4,gosh could find kat boyfriend,let see


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [11]:
questions = list()
for line in lines.questions:
    questions.append(line) 
    
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions) 
tokenized_questions = tokenizer.texts_to_sequences(questions) 

length_list = list()
for token_seq in tokenized_questions:
    length_list.append(len(token_seq))
max_question_length = np.array(length_list).max()
print( 'Input max length is {}'.format(max_question_length))

padded_questions = pad_sequences(tokenized_questions ,maxlen=max_question_length, padding='post' )
encoder_questions_data = np.array(padded_questions)
print( 'Encoder input data shape -> {}'.format( encoder_questions_data.shape ))

questions_word_dict = tokenizer.word_index
num_question_tokens = len( questions_word_dict )+1
print( 'Number of Input tokens = {}'.format(num_question_tokens))

Input max length is 20
Encoder input data shape -> (139409, 20)
Number of Input tokens = 25285


In [12]:
answers = list()
for line in lines.answers:
    answers.append('<START> ' + line + ' <END>')  

tokenizer = Tokenizer()
tokenizer.fit_on_texts( answers ) 
tokenized_answers = tokenizer.texts_to_sequences( answers ) 

length_list = list()
for token_seq in tokenized_answers:
    length_list.append( len( token_seq ))
max_answer_length = np.array( length_list ).max()
print( 'Output max length is {}'.format( max_answer_length ))

padded_answers = pad_sequences( tokenized_answers , maxlen=max_answer_length, padding='post' )
decoder_answers_data = np.array( padded_answers )
print( 'Decoder input data shape -> {}'.format( decoder_answers_data.shape ))

answers_word_dict = tokenizer.word_index
num_answer_tokens = len( answers_word_dict )+1
print( 'Number of Output tokens = {}'.format( num_answer_tokens))

Output max length is 25
Decoder input data shape -> (139409, 25)
Number of Output tokens = 24993


In [13]:
decoder_target_data = list()
for token_seq in tokenized_answers:
    decoder_target_data.append(token_seq[1: ]) 
    
padded_answers_lines = pad_sequences(decoder_target_data, maxlen=max_answer_length, padding='post')
decoder_target_data = np.array(padded_answers_lines)
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

Decoder target data shape -> (139409, 25)


In [14]:
import tensorflow as tf
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_question_tokens, 128 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 128 , return_state=True , recurrent_dropout=0.3 , 
                                                           dropout=0.5 )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_answer_tokens, 128 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 128 , return_state=True , return_sequences=True , recurrent_dropout=0.3 , 
                                    dropout=0.5)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_answer_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 128)    3236480     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 128)    3199104     ['input_2[0][0]']                
                                                                                              

In [15]:
model.fit([encoder_questions_data , decoder_answers_data], decoder_target_data,
          batch_size=512, epochs=200, verbose=1) 

Epoch 1/200
273/273 [==============================] - 186s 639ms/step - loss: 6.4881 - accuracy: 0.2080
Epoch 2/200
273/273 [==============================] - 132s 481ms/step - loss: 5.3860 - accuracy: 0.3051
Epoch 3/200
273/273 [==============================] - 128s 470ms/step - loss: 5.1075 - accuracy: 0.3626
Epoch 4/200
273/273 [==============================] - 125s 456ms/step - loss: 5.0576 - accuracy: 0.3632
Epoch 5/200
273/273 [==============================] - 125s 457ms/step - loss: 5.0253 - accuracy: 0.3639
Epoch 6/200
273/273 [==============================] - 125s 457ms/step - loss: 4.9989 - accuracy: 0.3644
Epoch 7/200
273/273 [==============================] - 125s 457ms/step - loss: 4.9773 - accuracy: 0.3647
Epoch 8/200
273/273 [==============================] - 124s 452ms/step - loss: 4.9589 - accuracy: 0.3666
Epoch 9/200
273/273 [==============================] - 124s 452ms/step - loss: 4.9427 - accuracy: 0.3670
Epoch 10/200
273/273 [==============================] -

In [20]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(None,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(None,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [46]:
import tensorflow as tf
def str_to_tokens( sentence : str ):
    sentence = clean_text(sentence)
    sentence = ' '.join([lemmatizer.lemmatize(word.lower(), pos="v") for word in sentence 
                             if not word.lower() in stop_words and word not in punctuation])
    words = sentence.split()
    tokens_list = list()
    for word in words:
        tokens_list.append( questions_word_dict[ word ] ) 
    return pad_sequences( [tokens_list] , maxlen=max_question_length , padding='post')

In [51]:
enc_model , dec_model = make_inference_models()
for epoch in range(8):
    states_values = enc_model.predict( str_to_tokens( input( 'User: ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = answers_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in answers_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_answer_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( "Bot:" +decoded_translation.replace(' end', '') )
    print()

User:  hi


1/1 [==============================] - 0s 22ms/step
Bot: yeah



User:  how are you?


1/1 [==============================] - 0s 23ms/step
Bot: oh right



User:  You're ready for the quiz.


...

1/1 [==============================] - 0s 28ms/step
Bot: yes sir



User:  You always been this selfish?


1/1 [==============================] - 0s 21ms/step
Bot: yes sure



User:  What crap?


1/1 [==============================] - 0s 21ms/step
Bot: five years ecological study northland oil

